<a href="https://colab.research.google.com/github/zamanmiraz/DSandML-Notebooks/blob/main/RAG/06_hybrid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/guyernest/advanced-rag.git
%cd advanced-rag
!pip install --upgrade -r requirements.txt

Cloning into 'advanced-rag'...
remote: Enumerating objects: 281, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 281 (delta 4), reused 10 (delta 3), pack-reused 268 (from 1)
Receiving objects: 100% (281/281), 18.84 MiB | 15.85 MiB/s, done.
Resolving deltas: 100% (167/167), done.
/content/advanced-rag
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.7/163.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install torchvision==0.18.0
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 738.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 688.5 kB/s eta 0:00:00
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.1.0.70
    Uninstalling nvidia-cudnn-cu12-9.1.0.70:
      Successfully uninstalled nvidia-cudnn-cu12-9.1.0.70
  Attempting uninstall: torch
    Found existing installation: torch 2.4.1
    Uninstalling torch-2.4.1:
      Successfully uninstalled torch-2.4.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.23.0+cu126
    Uninstalling torchvision-0.23.0+cu126:
      Successfully uninstalled torchvision-0.23.0+cu126
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.8.0+cu126 requires torch==2

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
^C


In [2]:
from rich.console import Console
from rich.style import Style
import pathlib
from rich_theme_manager import Theme, ThemeManager

THEMES = [
    Theme(
        name="dark",
        description="Dark mode theme",
        tags=["dark"],
        styles={
            "repr.own": Style(color="#e87d3e", bold=True),      # Class names
            "repr.tag_name": "dim cyan",                        # Adjust tag names
            "repr.call": "bright_yellow",                       # Function calls and other symbols
            "repr.str": "bright_green",                         # String representation
            "repr.number": "bright_red",                        # Numbers
            "repr.none": "dim white",                           # None
            "repr.attrib_name": Style(color="#e87d3e", bold=True),    # Attribute names
            "repr.attrib_value": "bright_blue",                 # Attribute values
            "default": "bright_white on black"                  # Default text and background
        },
    ),
    Theme(
        name="light",
        description="Light mode theme",
        styles={
            "repr.own": Style(color="#22863a", bold=True),          # Class names
            "repr.tag_name": Style(color="#00bfff", bold=True),     # Adjust tag names
            "repr.call": Style(color="#ffff00", bold=True),         # Function calls and other symbols
            "repr.str": Style(color="#008080", bold=True),          # String representation
            "repr.number": Style(color="#ff6347", bold=True),       # Numbers
            "repr.none": Style(color="#808080", bold=True),         # None
            "repr.attrib_name": Style(color="#ffff00", bold=True),  # Attribute names
            "repr.attrib_value": Style(color="#008080", bold=True), # Attribute values
            "default": Style(color="#000000", bgcolor="#ffffff"),   # Default text and background
        },
    ),
]

theme_dir = pathlib.Path("themes").expanduser()
theme_dir.expanduser().mkdir(parents=True, exist_ok=True)

theme_manager = ThemeManager(theme_dir=theme_dir, themes=THEMES)
theme_manager.list_themes()

dark = theme_manager.get("dark")
theme_manager.preview_theme(dark)
light = theme_manager.get("light")

console = Console(theme=light)

 Theme  Description       Tags  Path               
 dark   Dark mode theme   dark  themes/dark.theme  
 light  Light mode theme        themes/light.theme

                                      Theme: dark - themes/dark.theme                                      
┌───────────────────┬───────────────┬───────┬─────────┬─────────┬────────────────┬────────────────────────┐
│ style             │ color         │ color │ bgcolor │ bgcolor │ attributes     │ example                │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ default           │ bright_white  │ █████ │ black   │ █████   │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_name  │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.attrib_value │ bright_blue   │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.call         │ bright_yellow │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.none         │ white         │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.number       │ bright_red    │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.own          │ #e87d3e       │ █████ │ None    │         │ b------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.str          │ bright_green  │ █████ │ None    │         │ -------------- │ The quick brown fox... │
├───────────────────┼───────────────┼───────┼─────────┼─────────┼────────────────┼────────────────────────┤
│ repr.tag_name     │ cyan          │ █████ │ None    │         │ -d------------ │ The quick brown fox... │
└───────────────────┴───────────────┴───────┴─────────┴─────────┴────────────────┴────────────────────────┘
┌─ attributes legend ──────────────────────────────────────────────────────────────────┐
│  b: bold, d: dim, i: italic, u: underline, U: double underline, B: blink, 2: blink2  │
│  r: reverse, c: conceal, s: strike, f: frame, e: encircle, o: overline, L: Link      │
└──────────────────────────────────────────────────────────────────────────────────────┘

In [3]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

In [4]:
import bm25s
from bm25s.tokenization import Tokenizer, Tokenized
import Stemmer

In [5]:
import json
corpus_json = json.load(open('advanced-rag/data/corpus.json'))

In [6]:
corpus_text = [doc['text'] for doc in corpus_json]
# optional: create a stemmer
english_stemmer = Stemmer.Stemmer("english")
# Initialize the Tokenizer with the stemmer
sparse_tokenizer = Tokenizer(
    stemmer=english_stemmer,
    lower=True, # lowercase the tokens
    stopwords="english",  # or pass a list of stopwords
    splitter=r"\w+",  # by default r"(?u)\b\w\w+\b", can also be a function
)

In [7]:
console.print(sparse_tokenizer.stopwords)

(
    'a',
    'an',
    'and',
    'are',
    'as',
    'at',
    'be',
    'but',
    'by',
    'for',
    'if',
    'in',
    'into',
    'is',
    'it',
    'no',
    'not',
    'of',
    'on',
    'or',
    'such',
    'that',
    'the',
    'their',
    'then',
    'there',
    'these',
    'they',
    'this',
    'to',
    'was',
    'will',
    'with'
)

In [8]:
# Tokenize the corpus and only keep the ids (faster and saves memory)
corpus_sparse_tokens = (
    sparse_tokenizer
    .tokenize(
        corpus_text,
        update_vocab=True, # update the vocab as we tokenize
        return_as="ids"
    )
)

# Create the BM25 retriever and attach your corpus_json to it
sparse_index = bm25s.BM25(corpus=corpus_json)
# Now, index the corpus_tokens (the corpus_json is not used yet)
sparse_index.index(corpus_sparse_tokens)

Tokenize texts:   0%|          | 0/46 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from tokens


BM25S Create Vocab:   0%|          | 0/46 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/46 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/46 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/46 [00:00<?, ?it/s]

In [9]:
vocab_dict = sparse_tokenizer.get_vocab_dict()
console.print(f"The tokenizer vocabulary includes {len(vocab_dict)} tokens/terms")

focus_token = 'context'
focus_token_index = vocab_dict.get(focus_token)
console.print(f"The index of the {focus_token} is {focus_token_index}")

The tokenizer vocabulary includes 1689 tokens/terms

The index of the context is 127

In [10]:
console.print(sparse_tokenizer.decode([[focus_token_index]]))

[['context']]

In [11]:
console.print(sparse_index.scores)

{
    'data': array([0.738586  , 0.75115275, 1.0805568 , ..., 1.6571838 , 1.6571838 ,
       1.6571838 ], dtype=float32),
    'indices': array([ 0,  9, 10, ..., 45, 45, 45], dtype=int32),
    'indptr': array([   0,   12,   33, ..., 4037, 4038, 4039], dtype=int32),
    'num_docs': 46
}

In [12]:
from rich.table import Table
from rich.style import Style

token_index = vocab_dict.get(focus_token)
console.print(f"Index of the token `{focus_token}` in the BM25 retriever: {token_index}")
score_index = sparse_index.scores.get('indptr')[token_index]
next_score_index = sparse_index.scores.get('indptr')[token_index+1]

table = Table(title=f"Document Scores for `{focus_token}`")

table.add_column("Document ID", justify="right", style="cyan", no_wrap=True)
table.add_column("Score", justify="right", style="bright_green")

max_score = max(sparse_index.scores['data'][score_index:next_score_index])
# Define styles for specific rows
highlight_style = Style(bgcolor="yellow")

for i in range(score_index, next_score_index):
    doc_id = sparse_index.scores['indices'][i]
    doc_score = sparse_index.scores['data'][i]
    if doc_score == max_score:
        table.add_row(
            str(doc_id),
            str(doc_score), style=highlight_style
        )
    else:
        table.add_row(
            str(doc_id),
            str(doc_score)
        )

console.print(table)

Index of the token `context` in the BM25 retriever: 127

    Document Scores for     
         `context`          
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Document ID ┃      Score ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│           0 │  0.4434834 │
│           2 │  0.7355117 │
│           3 │ 0.53710693 │
│           4 │ 0.90847206 │
│          13 │  0.5116058 │
│          14 │  0.9670208 │
│          15 │  1.1056415 │
│          30 │  0.7444794 │
│          37 │  0.6708646 │
│          41 │  0.7355117 │
└─────────────┴────────────┘

In [13]:
# Query the corpus
query = "What is context size of Mixtral?"
query_tokens = (
    sparse_tokenizer
    .tokenize(
        [query],
        update_vocab=False,
        return_as="ids"
    )
)

console.print(query_tokens)

Tokenize texts:   0%|          | 0/1 [00:00<?, ?it/s]

[[127, 128, 15]]

In [14]:
# Query the corpus
sparse_results, sparse_scores = sparse_index.retrieve(query_tokens, k=10)

for i in range(sparse_results.shape[1]):
    doc, score = sparse_results[0, i], sparse_scores[0, i]
    console.print(f"Rank {i+1} (score: {score:.2f}): {doc}")

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Rank 1 (score: 1.99): {'id': 2, 'text': 'expertsâ ) to process the token and combine their output additively. This 
technique increases the number of parameters of a model while controlling cost and latency, as the model only uses 
a fraction of the total set of parameters per token. Mixtral is pretrained with multilingual data using a context 
size of 32k tokens. It either matches or exceeds the performance of Llama 2 70B and GPT-3.5, over several 
benchmarks. In particular, Mixture of Experts Layer i gating inputs af outputs router expert\n\nThis chunk 
describes the key architectural details of the Mixtral model, a sparse mixture-of-experts language model that 
outperforms larger models like Llama 2 70B and GPT-3.5 on various benchmarks.', 'metadata': {'title': 'Mixtral of 
Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

Rank 2 (score: 1.86): {'id': 14, 'text': "Active Params French Arc-c HellaS MMLU German Arc-c HellaS MMLU Spanish 
Arc-c HellaS MMLU Italian Arc-c HellaS MMLU 33B 70B 13B 42.9% 65.4% 49.0% 39.3% 68.1% 49.9% 49.9% 72.5% 64.3% 49.4%
70.9% 65.1% 58.2% 77.4% 70.9% 54.3% 73.0% 71.5% 55.4% 77.6% 72.5% 52.8% 75.1% 70.9% 41.1% 63.3% 48.7% 47.3% 68.7% 
64.2% 45.7% 69.8% 52.3% 50.5% 74.5% 66.0% Table 4: Comparison of Mixtral with Llama on Multilingual Benchmarks. On 
ARC Challenge, Hellaswag, and MMLU, Mixtral outperforms Llama 2 70B on 4 languages: French, German, Spanish, and 
Italian. # 3.2 Long range performance To assess the capabilities of Mixtral to tackle long context, we evaluate it 
on the passkey retrieval task introduced in [23], a synthetic task designed to measure the ability of the model to 
retrieve a passkey inserted randomly in a long prompt. Results in Figure 4 (Left) show that Mixtral achieves a 100%
retrieval accuracy regardless of the context length or the position of passkey in the sequence. Figure 4 (Right) 
shows that the perplexity of Mixtral on a subset of the proof-pile dataset [2] decreases monotonically as the size 
of the context increases. Passkey Performance ry 0.8 0.6 04 0.2 0.0 OK 4K 8K 12K 16K 20K 24K 28K Seq Len Passkey 
Loc\n\nThe chunk discusses Mixtral's performance on multilingual benchmarks and its ability to handle long-range 
context, demonstrating its strong capabilities in these areas.", 'metadata': {'title': 'Mixtral of Experts', 
'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

Rank 3 (score: 1.46): {'id': 1, 'text': 'chat model on human bench- marks. Both the base and instruct models are 
released under the Apache 2.0 license. Code: https://github.com/mistralai/mistral-src Webpage: 
https://mistral.ai/news/mixtral-of-experts/ # Introduction In this paper, we present Mixtral 8x7B, a sparse mixture
of experts model (SMoE) with open weights, licensed under Apache 2.0. Mixtral outperforms Llama 2 70B and GPT-3.5 
on most benchmarks. As it only uses a subset of its parameters for every token, Mixtral allows faster inference 
speed at low batch-sizes, and higher throughput at large batch-sizes. Mixtral is a sparse mixture-of-experts 
network. It is a decoder-only model where the feedforward block picks from a set of 8 distinct groups of 
parameters. At every layer, for every token, a router network chooses two of these groups (the â\n\nThis chunk 
introduces Mixtral 8x7B, a sparse mixture of experts language model that outperforms Llama 2 70B and GPT-3.5 on 
most benchmarks. It describes the key architectural details of Mixtral, including its use of a sparse 
mixture-of-experts network, and mentions that the base and instruct models are released under the Apache 2.0 
license.', 'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

Rank 4 (score: 1.33): {'id': 15, 'text': "3.8 â Mixtral_8x7B 3.5 32 > $3.0 i] 228 fos a 2.0 0 5k 10k 15k 20k 25k 
30k Context length Passkey Performance ry 3.8 â Mixtral_8x7B 3.5 0.8 32 > 0.6 $3.0 i] 228 04 fos 0.2 a 2.0 0.0 OK 
4K 8K 12K 16K 20K 24K 28K 0 5k 10k 15k 20k 25k 30k Seq Len Context length Figure 4: Long range performance of 
Mixtral. (Left) Mixtral has 100% retrieval accuracy of the Passkey task regardless of the location of the passkey 
and length of the input sequence. (Right) The perplexity of Mixtral on the proof-pile dataset decreases 
monotonically as the context length increases.\n\nThe chunk discusses the long-range performance of the Mixtral 
model, demonstrating its ability to retrieve a passkey regardless of its location in a long input sequence, and 
showing that the model's perplexity on the proof-pile dataset decreases as the context length increases.", 
'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

...

Rank 6 (score: 1.24): {'id': 12, 'text': 'Size and Efficiency. We compare our performance to the Llama 2 family, 
aiming to understand Mixtral modelsâ efficiency in the cost-performance spectrum (see Figure 3). As a sparse 
Mixture- of-Experts model, Mixtral only uses 13B active parameters for each token. With 5x lower active parameters,
Mixtral is able to outperform Llama 2 70B across most categories. Note that this analysis focuses on the active 
parameter count (see Section 2.1), which is directly proportional to the inference compute cost, but does not 
consider the memory costs and hardware utilization. The memory costs for serving Mixtral are proportional to its 
sparse parameter count, 47B, which is still smaller than Llama 2 70B. As for device utilization, we note that the 
SMoEs layer introduces additional overhead due to the routing mechanism and due to the increased memory loads when 
running more than one expert per device. They are more suitable for batched workloads where one can reach a good 
degree of arithmetic intensity. Comparison with Llama 2 70B and GPT-3.5. In Table 3, we report the performance of 
Mixtral 8x7B compared to Llama 2 70B and GPT-3.5. We observe that Mixtral performs similarly or above the two other
models. On MMLU, Mixtral obtains a better performance, despite its significantly smaller capacity (47B tokens 
compared to 70B). For MT Bench, we report the performance of the latest GPT-3.5-Turbo model available, 
gpt-3.5-turbo-1106. 2Since Llama 2 34B was not open-sourced, we report results for Llama 1 34B.\n\nThis chunk 
discusses the size and efficiency of the Mixtral model, comparing its performance to the Llama 2 family of models. 
It highlights that Mixtral, as a sparse mixture-of-experts model, uses significantly fewer active parameters than 
Llama 2 70B while outperforming it across most benchmarks. The chunk also compares the performance of Mixtral 8x7B 
to Llama 2 70B and GPT-3.5.', 'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': 
['1905.07830']}}

Rank 7 (score: 1.12): {'id': 4, 'text': "Instruct under the Apache 2.0 license1, free for academic and commercial 
usage, ensuring broad accessibility and potential for diverse applications. To enable the community to run Mixtral 
with a fully open-source stack, we submitted changes to the vLLM project, which integrates Megablocks CUDA kernels 
for efficient inference. Skypilot also allows the deployment of vLLM endpoints on any instance in the cloud. # 2 
Architectural details Mixtral is based on a transformer architecture [31] and uses the same modifications as 
described in [18], with the notable exceptions that Mix- tral supports a fully dense context length of 32k tokens, 
and the feed- forward blocks are replaced by Mixture-of-Expert layers (Section 2.1). The model architecture 
parameters are summarized in Table 1.\n\nThis chunk describes the architectural details of the Mixtral language 
model, including its use of a transformer architecture with a 32k token context length and mixture-of-expert 
layers. It also mentions the model's open-source licensing and deployment options.", 'metadata': {'title': 'Mixtral
of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

Rank 8 (score: 0.89): {'id': 41, 'text': 'Hellaswag: Can a machine really finish your sentence? arXiv preprint 
arXiv:1905.07830, 2019. [33] Lianmin Zheng, Wei-Lin Chiang, Ying Sheng, Siyuan Zhuang, Zhanghao Wu, Yonghao Zhuang,
Zi Lin, Zhuohan Li, Dacheng Li, Eric Xing, et al. Judging llm-as-a-judge with mt-bench and chatbot arena. arXiv 
preprint arXiv:2306.05685, 2023.\n\nThe chunk discusses two references related to language model benchmarking, 
including the Hellaswag dataset and the MT-Bench and Chatbot Arena benchmarks. This is situated within the broader 
context of the paper, which introduces the Mixtral language model and evaluates its performance on various 
benchmarks.', 'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

Rank 9 (score: 0.80): {'id': 3, 'text': 'Figure 1: Mixture of Experts Layer. Each input vector is assigned to 2 of 
the 8 experts by a router. The layerâ s output is the weighted sum of the outputs of the two selected experts. In 
Mixtral, an expert is a standard feedforward block as in a vanilla transformer architecture. Mixtral demonstrates 
superior capabilities in mathematics, code generation, and tasks that require multilingual understanding, 
significantly outperforming Llama 2 70B in these domains. Experiments show that Mixtral is able to successfully 
retrieve information from its context window of 32k tokens, regardless of the sequence length and the location of 
the information in the sequence. We also present Mixtral 8x7B â Instruct, a chat model fine-tuned to follow 
instructions using supervised fine-tuning and Direct Preference Optimization [25]. Its performance notably 
surpasses that of GPT-3.5 Turbo, Claude-2.1, Gemini Pro, and Llama 2 70B â chat model on human evaluation 
benchmarks. Mixtral â Instruct also demonstrates reduced biases, and a more balanced sentiment profile in 
benchmarks such as BBQ, and BOLD. We release both Mixtral 8x7B and Mixtral 8x7B â\n\nThis chunk describes the 
Mixture of Experts layer architecture used in the Mixtral model, as well as the superior performance of Mixtral 
compared to other models on various benchmarks, including mathematics, code generation, and multilingual tasks. It 
also introduces the Mixtral 8x7B - Instruct model, which is fine-tuned to follow instructions and outperforms other
chat models on human evaluation benchmarks.', 'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088',
'references': ['1905.07830']}}

Rank 10 (score: 0.75): {'id': 13, 'text': "4 LLaMA 2 70B GPT-3.5 MMLU (MCQ in 57 subjects) 69.9% 70.0% 70.6% 
HellaSwag (10-shot) 87.1% 85.5% 86.7% ARC Challenge (25-shot) 85.1% 85.2% 85.8% WinoGrande (5-shot) 83.2% 81.6% 
81.2% MBPP (pass@1) 49.8% 52.2% 60.7% GSM-8K (5-shot) 53.6% 57.1% 58.4% MT Bench (for Instruct Models) 6.86 8.32 
8.30 # Mixtral 8x7B Table 3: Comparison of Mixtral with Llama 2 70B and GPT-3.5. Mixtral outperforms or matches 
Llama 2 70B and GPT-3.5 performance on most metrics. Evaluation Differences. On some benchmarks, there are some 
differences between our evaluation protocol and the one reported in the Llama 2 paper: 1) on MBPP, we use the 
hand-verified subset 2) on TriviaQA, we do not provide Wikipedia contexts. # 3.1 Multilingual benchmarks Compared 
to Mistral 7B, we significantly upsample the proportion of multilingual data during pretraining. The extra capacity
allows Mixtral to perform well on multilingual benchmarks while maintaining a high accuracy in English. In 
particular, Mixtral significantly outperforms Llama 2 70B in French, German, Spanish, and Italian, as shown in 
Table 4.\n\nThis chunk presents a comparison of the performance of Mixtral 8x7B, Llama 2 70B, and GPT-3.5 on 
various benchmarks, as well as an analysis of Mixtral's performance on multilingual benchmarks.", 'metadata': 
{'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}}

In [15]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from sentence_transformers import SentenceTransformer

qdrant_client = QdrantClient(
    ":memory:"
)

# Create the embedding encoder
dense_encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [20]:
collection_name = "hybrid_search"

dense_index = qdrant_client.recreate_collection(
    collection_name=collection_name,
        vectors_config=models.VectorParams(
        size=dense_encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)
print(dense_index)

True


In [24]:
# vectorize!
qdrant_client.upload_points(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=idx,
            vector=dense_encoder.encode(doc["text"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(corpus_json) # data is the variable holding all the enriched texts
    ]
)

In [25]:
query_vector = dense_encoder.encode(query).tolist()

In [26]:
dense_results = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=10
)

In [27]:
console.print(dense_results)

[
    ScoredPoint(
        id=15,
        version=0,
        score=0.6180976543997941,
        payload={
            'id': 15,
            'text': "3.8 â Mixtral_8x7B 3.5 32 > $3.0 i] 228 fos a 2.0 0 5k 10k 15k 20k 25k 30k Context length 
Passkey Performance ry 3.8 â Mixtral_8x7B 3.5 0.8 32 > 0.6 $3.0 i] 228 04 fos 0.2 a 2.0 0.0 OK 4K 8K 12K 16K 20K 
24K 28K 0 5k 10k 15k 20k 25k 30k Seq Len Context length Figure 4: Long range performance of Mixtral. (Left) Mixtral
has 100% retrieval accuracy of the Passkey task regardless of the location of the passkey and length of the input 
sequence. (Right) The perplexity of Mixtral on the proof-pile dataset decreases monotonically as the context length
increases.\n\nThe chunk discusses the long-range performance of the Mixtral model, demonstrating its ability to 
retrieve a passkey regardless of its location in a long input sequence, and showing that the model's perplexity on 
the proof-pile dataset decreases as the context length increases.",
            'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
        },
        vector=None,
        shard_key=None,
        order_value=None
    ),
    ScoredPoint(
        id=4,
        version=0,
        score=0.49721741586617196,
        payload={
            'id': 4,
            'text': "Instruct under the Apache 2.0 license1, free for academic and commercial usage, ensuring broad
accessibility and potential for diverse applications. To enable the community to run Mixtral with a fully 
open-source stack, we submitted changes to the vLLM project, which integrates Megablocks CUDA kernels for efficient
inference. Skypilot also allows the deployment of vLLM endpoints on any instance in the cloud. # 2 Architectural 
details Mixtral is based on a transformer architecture [31] and uses the same modifications as described in [18], 
with the notable exceptions that Mix- tral supports a fully dense context length of 32k tokens, and the feed- 
forward blocks are replaced by Mixture-of-Expert layers (Section 2.1). The model architecture parameters are 
summarized in Table 1.\n\nThis chunk describes the architectural details of the Mixtral language model, including 
its use of a transformer architecture with a 32k token context length and mixture-of-expert layers. It also 
mentions the model's open-source licensing and deployment options.",
            'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
        },
        vector=None,
        shard_key=None,
        order_value=None
    ),
    ScoredPoint(
        id=2,
        version=0,
        score=0.44239128745163936,
        payload={
            'id': 2,
            'text': 'expertsâ ) to process the token and combine their output additively. This technique increases 
the number of parameters of a model while controlling cost and latency, as the model only uses a fraction of the 
total set of parameters per token. Mixtral is pretrained with multilingual data using a context size of 32k tokens.
It either matches or exceeds the performance of Llama 2 70B and GPT-3.5, over several benchmarks. In particular, 
Mixture of Experts Layer i gating inputs af outputs router expert\n\nThis chunk describes the key architectural 
details of the Mixtral model, a sparse mixture-of-experts language model that outperforms larger models like Llama 
2 70B and GPT-3.5 on various benchmarks.',
            'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
        },
        vector=None,
        shard_key=None,
        order_value=None
    ),
    ScoredPoint(
        id=6,
        version=0,
        score=0.4404993624382003,
        payload={
            'id': 6,
            'text': 'Table 1: Model architecture. # j nâ G(x)i Â· Ei(x). i=0 Here, G(x)i denotes the n-dimensional 
output of the gating network for the i-th expert, and Ei(x) is the output of the i-th expert network. 

In [28]:
documents_with_scores = []
for hit in dense_results:
    doc_id = hit.payload["id"]
    doc_text = next((doc for doc in corpus_json if doc["id"] == doc_id), None)["text"]
    doc_dense_score = hit.score
    documents_with_scores.append({
        "id": doc_id,
        "text": doc_text,
        "dense_score": doc_dense_score
    })

for i, result in enumerate(sparse_results[0]):
    doc_id = result["id"]
    doc_text = next((doc for doc in corpus_json if doc["id"] == doc_id), None)["text"]
    doc_sparse_score = sparse_scores[0][i]
    for doc in documents_with_scores:
        if doc["id"] == doc_id:
            doc["sparse_score"] = doc_sparse_score
            break

In [29]:
console.print(documents_with_scores)

[
    {
        'id': 15,
        'text': "3.8 â Mixtral_8x7B 3.5 32 > $3.0 i] 228 fos a 2.0 0 5k 10k 15k 20k 25k 30k Context length Passkey 
Performance ry 3.8 â Mixtral_8x7B 3.5 0.8 32 > 0.6 $3.0 i] 228 04 fos 0.2 a 2.0 0.0 OK 4K 8K 12K 16K 20K 24K 28K 0 
5k 10k 15k 20k 25k 30k Seq Len Context length Figure 4: Long range performance of Mixtral. (Left) Mixtral has 100% 
retrieval accuracy of the Passkey task regardless of the location of the passkey and length of the input sequence. 
(Right) The perplexity of Mixtral on the proof-pile dataset decreases monotonically as the context length 
increases.\n\nThe chunk discusses the long-range performance of the Mixtral model, demonstrating its ability to 
retrieve a passkey regardless of its location in a long input sequence, and showing that the model's perplexity on 
the proof-pile dataset decreases as the context length increases.",
        'dense_score': 0.6180976543997941,
        'sparse_score': 1.333729
    },
    {
        'id': 4,
        'text': "Instruct under the Apache 2.0 license1, free for academic and commercial usage, ensuring broad 
accessibility and potential for diverse applications. To enable the community to run Mixtral with a fully 
open-source stack, we submitted changes to the vLLM project, which integrates Megablocks CUDA kernels for efficient
inference. Skypilot also allows the deployment of vLLM endpoints on any instance in the cloud. # 2 Architectural 
details Mixtral is based on a transformer architecture [31] and uses the same modifications as described in [18], 
with the notable exceptions that Mix- tral supports a fully dense context length of 32k tokens, and the feed- 
forward blocks are replaced by Mixture-of-Expert layers (Section 2.1). The model architecture parameters are 
summarized in Table 1.\n\nThis chunk describes the architectural details of the Mixtral language model, including 
its use of a transformer architecture with a 32k token context length and mixture-of-expert layers. It also 
mentions the model's open-source licensing and deployment options.",
        'dense_score': 0.49721741586617196,
        'sparse_score': 1.1242076
    },
    {
        'id': 2,
        'text': 'expertsâ ) to process the token and combine their output additively. This technique increases the 
number of parameters of a model while controlling cost and latency, as the model only uses a fraction of the total 
set of parameters per token. Mixtral is pretrained with multilingual data using a context size of 32k tokens. It 
either matches or exceeds the performance of Llama 2 70B and GPT-3.5, over several benchmarks. In particular, 
Mixture of Experts Layer i gating inputs af outputs router expert\n\nThis chunk describes the key architectural 
details of the Mixtral model, a sparse mixture-of-experts language model that outperforms larger models like Llama 
2 70B and GPT-3.5 on various benchmarks.',
        'dense_score': 0.44239128745163936,
        'sparse_score': 1.9919165
    },
    {
        'id': 6,
        'text': 'Table 1: Model architecture. # j nâ G(x)i Â· Ei(x). i=0 Here, G(x)i denotes the n-dimensional 
output of the gating network for the i-th expert, and Ei(x) is the output of the i-th expert network. If the gating
vector is sparse, we can avoid computing the outputs of experts whose gates are zero. There are multiple 
alternative ways of implementing G(x) [6, 15, 35], but a simple and performant one is implemented by taking the 
softmax over the Top-K logits of a linear layer [28].\n\nThe chunk describes the architectural details of the 
Mixtral model, specifically the Sparse Mixture of Experts (SMoE) layer that is used in the model.',
        'dense_score': 0.4404993624382003
    },
    {
        'id': 7,
        'text': 'We use G(x) := Softmax(TopK(x Â· Wg)), where (TopK(â ))i := â i if â i is among the top-K 
coordinates of logits â â Rn and (TopK(â ))i := â â otherwise. The value of K â the number of experts used per 
token â is a hyper-paramet

In [30]:
import numpy as np

# Normalize the two types of scores
dense_scores = np.array([doc.get("dense_score", 0) for doc in documents_with_scores])
sparse_scores = np.array([doc.get("sparse_score", 0) for doc in documents_with_scores])

dense_scores_normalized = (dense_scores - np.min(dense_scores)) / (np.max(dense_scores) - np.min(dense_scores))
sparse_scores_normalized = (sparse_scores - np.min(sparse_scores)) / (np.max(sparse_scores) - np.min(sparse_scores))

# Calculate a weighted score with alpha of 0.2 to the sparse score
alpha = 0.2
weighted_scores = (1 - alpha) * dense_scores_normalized + alpha * sparse_scores_normalized

# Pick up the top 3 documents with the weighted score
top_docs = sorted(
    zip(
        documents_with_scores,
        weighted_scores
    ),
    key=lambda x: x[1],
    reverse=True
)[:3]

In [31]:
console.print(top_docs)

[
    (
        {
            'id': 15,
            'text': "3.8 â Mixtral_8x7B 3.5 32 > $3.0 i] 228 fos a 2.0 0 5k 10k 15k 20k 25k 30k Context length 
Passkey Performance ry 3.8 â Mixtral_8x7B 3.5 0.8 32 > 0.6 $3.0 i] 228 04 fos 0.2 a 2.0 0.0 OK 4K 8K 12K 16K 20K 
24K 28K 0 5k 10k 15k 20k 25k 30k Seq Len Context length Figure 4: Long range performance of Mixtral. (Left) Mixtral
has 100% retrieval accuracy of the Passkey task regardless of the location of the passkey and length of the input 
sequence. (Right) The perplexity of Mixtral on the proof-pile dataset decreases monotonically as the context length
increases.\n\nThe chunk discusses the long-range performance of the Mixtral model, demonstrating its ability to 
retrieve a passkey regardless of its location in a long input sequence, and showing that the model's perplexity on 
the proof-pile dataset decreases as the context length increases.",
            'dense_score': 0.6180976543997941,
            'sparse_score': 1.333729
        },
        0.9339141478808913
    ),
    (
        {
            'id': 4,
            'text': "Instruct under the Apache 2.0 license1, free for academic and commercial usage, ensuring broad
accessibility and potential for diverse applications. To enable the community to run Mixtral with a fully 
open-source stack, we submitted changes to the vLLM project, which integrates Megablocks CUDA kernels for efficient
inference. Skypilot also allows the deployment of vLLM endpoints on any instance in the cloud. # 2 Architectural 
details Mixtral is based on a transformer architecture [31] and uses the same modifications as described in [18], 
with the notable exceptions that Mix- tral supports a fully dense context length of 32k tokens, and the feed- 
forward blocks are replaced by Mixture-of-Expert layers (Section 2.1). The model architecture parameters are 
summarized in Table 1.\n\nThis chunk describes the architectural details of the Mixtral language model, including 
its use of a transformer architecture with a 32k token context length and mixture-of-expert layers. It also 
mentions the model's open-source licensing and deployment options.",
            'dense_score': 0.49721741586617196,
            'sparse_score': 1.1242076
        },
        0.5466318985315719
    ),
    (
        {
            'id': 2,
            'text': 'expertsâ ) to process the token and combine their output additively. This technique increases 
the number of parameters of a model while controlling cost and latency, as the model only uses a fraction of the 
total set of parameters per token. Mixtral is pretrained with multilingual data using a context size of 32k tokens.
It either matches or exceeds the performance of Llama 2 70B and GPT-3.5, over several benchmarks. In particular, 
Mixture of Experts Layer i gating inputs af outputs router expert\n\nThis chunk describes the key architectural 
details of the Mixtral model, a sparse mixture-of-experts language model that outperforms larger models like Llama 
2 70B and GPT-3.5 on various benchmarks.',
            'dense_score': 0.44239128745163936,
            'sparse_score': 1.9919165
        },
        0.46764174640365863
    )
]

In [32]:
# define a variable to hold the search results for the generation model
search_results = [doc[0]['text'] for doc in top_docs]

In [37]:
# Now time to connect to the large language model
import google.generativeai as genai
from google.colab import userdata
from rich.text import Text

# Configure the Gemini API
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Initialize the Generative Model
# You can choose a different model if needed
gemini_model = genai.GenerativeModel('gemini-2.5-flash')

# Create the prompt for the Gemini model
prompt = f"""You are chatbot, an research expert. Your top priority is to help guide users to understand research papers.

Based on the following search results, answer the user's query:

Search results:
{search_results}

User query:
{query}
"""

# Generate the response using the Gemini model
response = gemini_model.generate_content(prompt)
response_text = Text(response.text)

# Display the response (optional, you might want to print it directly)
# console.print(response_text)

In [38]:
from rich.panel import Panel

panel = Panel(response_text, title=f"Hybrid Search Reply to \"{query}\"")
console.print(panel)

╭─────────────────────────── Hybrid Search Reply to "What is context size of Mixtral?" ───────────────────────────╮
│ Based on the search results, Mixtral supports a fully dense context length of **32k tokens**. It was also       │
│ pretrained using this 32k token context size.                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯